In [51]:
import sys
try:
    import pandas as pd
except ImportError as e:
    print("pandas is required for this example. Please install with conda or pip  and then try again.")
    sys.exit()

import numpy as np
import sklearn
from sklearn import ensemble
import kmapper as km
from kmapper.plotlyviz import *
import plotly.graph_objs as go
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
pd.set_option('display.float_format', lambda x: '%.2f' % x)

import warnings
warnings.filterwarnings("ignore")

In [4]:
# Data - the Wisconsin Breast Cancer Dataset
# https://www.kaggle.com/uciml/breast-cancer-wisconsin-data
path = r'C:\Users\Doug\Documents\Classes\TDA\Data\Predict_summer_13_backwater.csv'
df = pd.read_csv(path)
df = df[["WDP", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"]]
feature_names = [c for c in df.columns]
X = np.array(df[feature_names].fillna(0))

# Create a custom 1-D lens with Isolation Forest
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0], 1))

# Create another 1-D lens with L2-norm
mapper = km.KeplerMapper(verbose=0)
lens2 = mapper.fit_transform(X, projection="l2norm")

# Combine both lenses to get a 2-D [Isolation Forest, L^2-Norm] lens
lens = np.c_[lens1, lens2]

# # Define the simplicial complex
# scomplex = mapper.map(lens,
#                       X,
#                       clusterer=sklearn.cluster.KMeans(n_clusters=2,
#                                                        random_state=3471))
# Define the simplicial complex
# Change cover; kmapper.map.cover shouldn't be None
scomplex = mapper.map(lens,
                      X,
                      cover= km.Cover(n_cubes=[10,10], perc_overlap=[0.33,0.33]),
                      clusterer=sklearn.cluster.DBSCAN(eps=10, min_samples=2, metric='euclidean'))


pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

color_values = lens [:,0] - lens[:,0].min()
my_colorscale = pl_brewer
kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                 color_values,  
                                                                 color_function_name='Distance to x-min', 
                                                                 colorscale=my_colorscale)

# assign to node['custom_tooltips']  the node label (0 for benign, 1 for malignant)
# for node in kmgraph['nodes']:
#     node['custom_tooltips'] = y[scomplex['nodes'][node['name']]]
    
bgcolor = 'rgba(10,10,10, 0.9)'
# y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                 factor_size=2.5, edge_linewidth=0.5)
layout = plot_layout(title='Topological network representing pool 13, summer',  
                     width=620, height=570,
                     annotation_text=get_kmgraph_meta(mapper_summary),  
                     bgcolor=bgcolor)

fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
fw_summary = summary_fig(mapper_summary, height=300)
dashboard = hovering_widgets(kmgraph, 
                             fw_graph, 
                             bgcolor=bgcolor, 
                             member_textbox_width=600)

#Update the fw_graph colorbar, setting its title:
fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'

In [36]:
dashboard

    'data': [{'hoverinfo': 'none',
              'line': {'color'…

In [2]:
path = r'C:\Users\Doug\Documents\Classes\TDA\Data\Predict_summer_13_backwater.csv'
df = pd.read_csv(path)
df = df[["DATE", "WDP", "STRATUM","DO","COND","VEL", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"]]
feature_names = [c for c in df.columns]
X = np.array(df[feature_names].fillna(0))
df['YEAR'] = pd.DatetimeIndex(df['DATE']).year

In [2]:
feature_names = [c for c in df.columns]
X = np.array(df[feature_names].fillna(0))
model = ensemble.IsolationForest(random_state=1729)
model.fit(X)
lens1 = model.decision_function(X).reshape((X.shape[0], 1))

NameError: name 'df' is not defined

In [8]:
df['YEAR'].unique()

array([1993, 1994, 1995, 1997, 1998, 1999, 2000, 2001, 2002, 2004, 2005,
       2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016,
       2017, 2018, 2019], dtype=int64)

In [2]:
# All years in the 1990's
time_1990 = [1990,1991,1992, 1993, 1994, 1995, 1997, 1998, 1999]
df_1990 = df[df['YEAR'].isin(time_1990)].drop(['YEAR', 'DATE'], axis=1)
df_1990.name = 'df_1990'
# All years in the 2000's
time_2000 = [2000, 2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009]
df_2000 = df[df['YEAR'].isin(time_2000)].drop(['YEAR', 'DATE'], axis=1)
df_2000.name = 'df_2000'
# All years in the 2010's
time_2010 = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_2010 = df[df['YEAR'].isin(time_2010)].drop(['YEAR', 'DATE'], axis=1)
df_2010.name = 'df_2010'
print(f"{df_1990.shape=}\n{df_2000.shape=}\n{df_2010.shape=}")

NameError: name 'df' is not defined

In [64]:
df_1990.name

'df_1990'

In [5]:
# Define parameters for Kepler Mapper; nr_cubes, perc_overlap
kmapper_params = [{"n_cubes": [7,7],"perc_overlap": [.30,.30]},
                  {"n_cubes": [7,10],"perc_overlap": [.30,.30]},
                  {"n_cubes": [7,13],"perc_overlap": [.30,.30]},
                  {"n_cubes": [10,7],"perc_overlap": [.30,.30]},
                  {"n_cubes": [10,10],"perc_overlap": [.30,.30]},
                  {"n_cubes": [10,13],"perc_overlap": [.30,.30]},
                  {"n_cubes": [13,7],"perc_overlap": [.30,.30]},
                  {"n_cubes": [13,10],"perc_overlap": [.30,.30]},
                  {"n_cubes": [13,13],"perc_overlap": [.30,.30]},
                  {"n_cubes": [7,7],"perc_overlap": [.35,.35]},
                  {"n_cubes": [7,10],"perc_overlap": [.35,.35]},
                  {"n_cubes": [7,13],"perc_overlap": [.35,.35]},
                  {"n_cubes": [10,7],"perc_overlap": [.35,.35]},
                  {"n_cubes": [10,10],"perc_overlap": [.35,.35]},
                  {"n_cubes": [10,13],"perc_overlap": [.35,.35]},
                  {"n_cubes": [13,7],"perc_overlap": [.35,.35]},
                  {"n_cubes": [13,10],"perc_overlap": [.35,.35]},
                  {"n_cubes": [13,13],"perc_overlap": [.35,.35]},
                  {"n_cubes": [7,7],"perc_overlap": [.40,.40]},
                  {"n_cubes": [7,10],"perc_overlap": [.40,.40]},
                  {"n_cubes": [7,13],"perc_overlap": [.40,.40]},
                  {"n_cubes": [10,7],"perc_overlap": [.40,.40]},
                  {"n_cubes": [10,10],"perc_overlap": [.40,.40]},
                  {"n_cubes": [10,13],"perc_overlap": [.40,.40]},
                  {"n_cubes": [13,7],"perc_overlap": [.40,.40]},
                  {"n_cubes": [13,10],"perc_overlap": [.40,.40]},
                  {"n_cubes": [13,13],"perc_overlap": [.40,.40]},
                  {"n_cubes": [7,7],"perc_overlap": [.45,.45]},
                  {"n_cubes": [7,10],"perc_overlap": [.45,.45]},
                  {"n_cubes": [7,13],"perc_overlap": [.45,.45]},
                  {"n_cubes": [10,7],"perc_overlap": [.45,.45]},
                  {"n_cubes": [10,10],"perc_overlap": [.45,.45]},
                  {"n_cubes": [10,13],"perc_overlap": [.45,.45]},
                  {"n_cubes": [13,7],"perc_overlap": [.45,.45]},
                  {"n_cubes": [13,10],"perc_overlap": [.45,.45]},
                  {"n_cubes": [13,13],"perc_overlap": [.45,.45]}]

In [2]:
decade_data = [df_1990, df_2000, df_2010]
eps_values = [1,5,10,15,20]
# feature_names = [c for c in df.columns]
# X = np.array(df_1990[feature_names].fillna(0))
# # Create a custom 1-D lens with Isolation Forest
# model = ensemble.IsolationForest(random_state=1729)
# model.fit(X)
# lens1 = model.decision_function(X).reshape((X.shape[0], 1))

# # Create another 1-D lens with L2-norm
# mapper = km.KeplerMapper(verbose=0)
# lens2 = mapper.fit_transform(X, projection="l2norm")
# # Combine both lenses to get a 2-D [Isolation Forest, L^2-Norm] lens
# lens = np.c_[lens1, lens2]


for j in decade_data:
    feature_names = [c for c in j.columns]
    X = np.array(j[feature_names].fillna(0))
    # Create a custom 1-D lens with Isolation Forest
    pca = sklearn.decomposition.PCA(n_components=2)
    mapper = km.KeplerMapper(verbose=0)
    lens = pca.fit_transform(X)
    for i in kmapper_params:
        feature_names = [c for c in j.columns]
        X = np.array(j[feature_names].fillna(0))
        scomplex = mapper.map(lens,X, cover= km.Cover(n_cubes=[7,7], perc_overlap=[.4,.4]),
                            clusterer=sklearn.cluster.DBSCAN(eps=10, min_samples=1, metric='euclidean'))
        pl_brewer = [[0.0, '#006837'],
                     [0.1, '#1a9850'],
                     [0.2, '#66bd63'],
                     [0.3, '#a6d96a'],
                     [0.4, '#d9ef8b'],
                     [0.5, '#ffffbf'],
                     [0.6, '#fee08b'],
                     [0.7, '#fdae61'],
                     [0.8, '#f46d43'],
                     [0.9, '#d73027'],
                     [1.0, '#a50026']]

        color_values = lens [:,0] - lens[:,0].min()
        my_colorscale = pl_brewer
        kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                         color_values,  
                                                                         color_function_name='Distance to x-min', 
                                                                         colorscale=my_colorscale)

        # assign to node['custom_tooltips']  the node label (0 for benign, 1 for malignant)
        # for node in kmgraph['nodes']:
        #     node['custom_tooltips'] = y[scomplex['nodes'][node['name']]]

        bgcolor = 'rgba(10,10,10, 0.9)'
        # y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

        plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                         factor_size=2.5, edge_linewidth=0.5)
        layout = plot_layout(title='Topological network representing pool 13, summer',  
                             width=620, height=570,
                             annotation_text=get_kmgraph_meta(mapper_summary),  
                             bgcolor=bgcolor)

        fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
        fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
        fw_summary = summary_fig(mapper_summary, height=300)
        dashboard = hovering_widgets(kmgraph, 
                                     fw_graph, 
                                     bgcolor=bgcolor, 
                                     member_textbox_width=600)
        #Update the fw_graph colorbar, setting its title:
        fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'
        output_path = r"C:\Users\Doug\Documents\Classes\TDA\kmapper_output_03242021\\" + str(j.name) + '_nCubes' + str(i['n_cubes'][0]) + str(i['n_cubes'][1]) + '_percOverlap' + str(i['perc_overlap'][0]) + ' ' + str(i['perc_overlap'][1]) + '.png'
        fw_graph.write_image(output_path)

NameError: name 'df_1990' is not defined

## Generate html for 03/24/2021

In [53]:
# Load data
path = r'C:\Users\Doug\Documents\Classes\TDA\Data\Predict_summer_13_backwater.csv'
df = pd.read_csv(path)
df = df[["DATE", "WDP", "STRATUM","DO","COND","VEL", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"]]
feature_names = [c for c in df.columns]
# X = np.array(df[feature_names].fillna(0))
df['YEAR'] = pd.DatetimeIndex(df['DATE']).year
# All years in the 1990's
time_1990 = [1990,1991,1992, 1993, 1994, 1995, 1997, 1998, 1999]
df_1990 = df[df['YEAR'].isin(time_1990)].drop(['YEAR', 'DATE'], axis=1)
df_1990.name = 'df_1990'
# All years in the 2000's
time_2000 = [2000, 2001, 2002,2003, 2004, 2005, 2006, 2007, 2008, 2009]
df_2000 = df[df['YEAR'].isin(time_2000)].drop(['YEAR', 'DATE'], axis=1)
df_2000.name = 'df_2000'
# All years in the 2010's
time_2010 = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]
df_2010 = df[df['YEAR'].isin(time_2010)].drop(['YEAR', 'DATE'], axis=1)
df_2010.name = 'df_2010'
print(f"{df_1990.shape=}\n{df_2000.shape=}\n{df_2010.shape=}")


# GLOBALS
EPSILON = 30
MIN_SAMPLES = 1
N_CUBES = [5,5]
PERC_OVERLAP = [.35,.35]

# 'PredictedTP', 'CHLcal', 'TURB'
# Narrow data to PredictedTP, CHLcal, TURB
# X = np.array(df_1990[["WDP", "STRATUM","DO","COND","VEL", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"]].fillna(0))
X = np.array(df_1990[['PredictedTP', 'CHLcal', 'TURB']].fillna(0))
pca = sklearn.decomposition.PCA(n_components=2)
lens = pca.fit_transform(X)
mapper = km.KeplerMapper(verbose=0)
feature_names = [c for c in df_1990.columns]
scomplex = mapper.map(lens,X, cover= km.Cover(n_cubes=N_CUBES, perc_overlap=PERC_OVERLAP),
                    clusterer=sklearn.cluster.DBSCAN(eps=EPSILON, min_samples=MIN_SAMPLES,
                                                     metric='euclidean'))
pl_brewer = [[0.0, '#006837'],
             [0.1, '#1a9850'],
             [0.2, '#66bd63'],
             [0.3, '#a6d96a'],
             [0.4, '#d9ef8b'],
             [0.5, '#ffffbf'],
             [0.6, '#fee08b'],
             [0.7, '#fdae61'],
             [0.8, '#f46d43'],
             [0.9, '#d73027'],
             [1.0, '#a50026']]

color_values = lens [:,0] - lens[:,0].min()
my_colorscale = pl_brewer
kmgraph,  mapper_summary, colorf_distribution = get_mapper_graph(scomplex, 
                                                                 color_values,  
                                                                 color_function_name='Distance to x-min', 
                                                                 colorscale=my_colorscale)

# assign to node['custom_tooltips']  the node label (0 for benign, 1 for malignant)
# for node in kmgraph['nodes']:
#     node['custom_tooltips'] = y[scomplex['nodes'][node['name']]]

bgcolor = 'rgba(10,10,10, 0.9)'
# y_gridcolor = 'rgb(150,150,150)'# on a black background the gridlines are set on  grey

plotly_graph_data = plotly_graph(kmgraph, graph_layout='fr', colorscale=my_colorscale, 
                                 factor_size=2.5, edge_linewidth=0.5)
plot_title = 'Pool 13, Summer 1993-1999; Epsilon ' + str(EPSILON) + ', MIN_SAMPLES ' + str(MIN_SAMPLES) 
layout = plot_layout(title=plot_title,  
                     width=620, height=570,
                     annotation_text=get_kmgraph_meta(mapper_summary),  
                     bgcolor=bgcolor)

# FigureWidget is responsible for event listeners

fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
fw_summary = summary_fig(mapper_summary, height=300)

dashboard = hovering_widgets(kmgraph, 
                             fw_graph, 
                             bgcolor=bgcolor, 
                             member_textbox_width=600)

#Update the fw_graph colorbar, setting its title:
fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'

df_1990.shape=(334, 12)
df_2000.shape=(529, 12)
df_2010.shape=(592, 12)


In [54]:
html_output_path = r"C:\Users\Doug\Documents\Classes\TDA\\" + 'all_1990s_epsilon' + str(EPSILON) + '_minsamples' + str(MIN_SAMPLES) + '_d04072021' + '.html'
mapper.visualize(scomplex, path_html=html_output_path)

'<!DOCTYPE html>\n<html>\n\n<head>\n  <meta charset="utf-8">\n  <meta name="generator" content="KeplerMapper">\n  <title>Kepler Mapper | KeplerMapper</title>\n\n  <link rel="icon" type="image/png" href="http://i.imgur.com/axOG6GJ.jpg" />\n\n  <link href=\'https://fonts.googleapis.com/css?family=Roboto+Mono:700,300\' rel=\'stylesheet\' type=\'text/css\'>\n  <style>* {\n  margin: 0;\n  padding: 0;\n}\n\nhtml, body {\n  height: 100%;\n}\n\nbody {\n  font-family: "Roboto Mono", "Helvetica", sans-serif;\n  font-size: 14px;\n}\n\n#logo {\n  position: absolute;\n  right: 00px;\n  top: 0px;\n  width: 90px;\n  height: 90px;\n  z-index: 999999;\n}\n\n#display {\n  color: #95A5A6;\n  background: #212121;\n}\n\n#print {\n  color: #000;\n  background: #FFF;\n}\n\nh1 {\n  font-size: 21px;\n  font-weight: 300;\n  font-weight: 300;\n}\n\nh2 {\n  font-size: 18px;\n  padding-bottom: 20px;\n  font-weight: 300;\n}\n\nh3 {\n  font-size: 14px;\n  font-weight: 700;\n  text-transform: uppercase;\n}\n\nh4 {\n 

In [25]:
dashboard

    'data': [{'hoverinfo': 'none',
              'line': {'color'…

In [23]:
vertex_stats = []
iter = 0

# Iterate over all nodes
while iter < len(kmgraph['nodes']):
    cluster_name = str(kmgraph['nodes'][iter]['name'])
    member_indices_of_vertex = kmgraph['nodes'][iter]['member_ids']
#     values_of_cluster = pd.DataFrame(columns=['PredictedTP', 'CHLcal', 'TURB'])
    values_of_cluster = pd.DataFrame(columns=["WDP", "STRATUM","DO","COND","VEL", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"])
    for member in member_indices_of_vertex:
        values_of_cluster = values_of_cluster.append(pd.DataFrame(X[member].reshape(1,-1),
                                                                  columns=list(values_of_cluster)), ignore_index=True)
    values_of_cluster['cluster_name'] = cluster_name
    vertex_stats.append(values_of_cluster)
    iter += 1

## Three variables summary, 1990s

In [56]:
vertex_stats = []
iter = 0

# Iterate over all nodes
while iter < len(kmgraph['nodes']):
    cluster_name = str(kmgraph['nodes'][iter]['name'])
    member_indices_of_vertex = kmgraph['nodes'][iter]['member_ids']
#     values_of_cluster = pd.DataFrame(columns=['PredictedTP', 'CHLcal', 'TURB'])
    values_of_cluster = pd.DataFrame(columns=['PredictedTP', 'CHLcal', 'TURB'])
    for member in member_indices_of_vertex:
        values_of_cluster = values_of_cluster.append(pd.DataFrame(X[member].reshape(1,-1),
                                                                  columns=list(values_of_cluster)), ignore_index=True)
    values_of_cluster['cluster_name'] = cluster_name
    vertex_stats.append(values_of_cluster)
    iter += 1
    
desired_km_vertices = ['cube9_cluster0', 'cube8_cluster0', 'cube13_cluster0', 'cube14_cluster0',
                     'cube15_cluster0', 'cube10_cluster0', 'cube5_cluster0', 'cube1_cluster0',
                       'cube19_cluster0', 'cube0_cluster0']
summary = pd.DataFrame()
for i in vertex_stats:
    if i.cluster_name.any() in desired_km_vertices:
        summary = summary.append(i)

temp = summary[['TURB', 'CHLcal', 'PredictedTP']]
print(len()'variables, 1990s \n EPSILON = ' + str(EPSILON) + '\n MIN_SAMPLES = ' + str(MIN_SAMPLES) + '\n N_CUBES = ' + str(N_CUBES) + '\n PERC_OVERLAP = ' + str(PERC_OVERLAP) + '\n')
print(temp.describe())

All variables, 1990s 
 EPSILON = 30
 MIN_SAMPLES = 1
 N_CUBES = [5, 5]
 PERC_OVERLAP = [0.35, 0.35]

        TURB  CHLcal  PredictedTP
count 440.00  440.00       440.00
mean   25.69   37.02         0.34
std    13.29   26.72         0.24
min     1.00    0.71         0.01
25%    16.00   17.61         0.19
50%    25.00   30.34         0.25
75%    35.00   45.71         0.40
max    84.00  122.85         1.35


## All variables summary, 1990s

In [52]:
vertex_stats = []
iter = 0

# Iterate over all nodes
while iter < len(kmgraph['nodes']):
    cluster_name = str(kmgraph['nodes'][iter]['name'])
    member_indices_of_vertex = kmgraph['nodes'][iter]['member_ids']
#     values_of_cluster = pd.DataFrame(columns=['PredictedTP', 'CHLcal', 'TURB'])
    values_of_cluster = pd.DataFrame(columns=["WDP", "STRATUM","DO","COND","VEL", "SECCHI", "TURB", "SS", "CHLcal", "TEMP", "PredictedTN", "PredictedTP"])
    for member in member_indices_of_vertex:
        values_of_cluster = values_of_cluster.append(pd.DataFrame(X[member].reshape(1,-1),
                                                                  columns=list(values_of_cluster)), ignore_index=True)
    values_of_cluster['cluster_name'] = cluster_name
    vertex_stats.append(values_of_cluster)
    iter += 1
    
desired_km_vertices = ['cube9_cluster0', 'cube8_cluster0', 'cube13_cluster0', 'cube14_cluster0',
                     'cube15_cluster0', 'cube10_cluster0', 'cube5_cluster0', 'cube1_cluster0',
                       'cube19_cluster0', 'cube0_cluster0']
summary = pd.DataFrame()
for i in vertex_stats:
    if i.cluster_name.any() in desired_km_vertices:
        summary = summary.append(i)

temp = summary[['TURB', 'CHLcal', 'PredictedTP']]
print('All variables, 1990s \nEPSILON = 30 \nMIN_SAMPLES = 1 \nN_CUBES = [5,5] \nPERC_OVERLAP = [.35,.35]\n')
print(temp.describe())

All variables, 1990s 
EPSILON = 30 
MIN_SAMPLES = 1 
N_CUBES = [5,5] 
PERC_OVERLAP = [.35,.35]

        TURB  CHLcal  PredictedTP
count 685.00  685.00       685.00
mean   29.34   31.49         0.31
std    14.21   22.97         0.20
min     1.00    1.79         0.01
25%    20.00   14.76         0.19
50%    29.00   23.93         0.24
75%    39.00   40.87         0.33
max    83.00  107.88         1.06


In [50]:
df_1990.shape

(334, 12)

In [13]:
layout = plot_layout(title=plot_title,  
                     width=620, height=570,
                     annotation_text=get_kmgraph_meta(mapper_summary),  
                     bgcolor=bgcolor)

# FigureWidget is responsible for event listeners

fw_graph = go.FigureWidget(data=plotly_graph_data, layout=layout)
fw_hist = node_hist_fig(colorf_distribution, bgcolor=bgcolor)
fw_summary = summary_fig(mapper_summary, height=300)

dashboard = hovering_widgets(kmgraph, 
                             fw_graph, 
                             bgcolor=bgcolor, 
                             member_textbox_width=600)

#Update the fw_graph colorbar, setting its title:
fw_graph.data[1].marker.colorbar.title = 'dist to<br>x-min'